In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
df = pd.read_csv('./total_data.csv', encoding = 'cp949')
df.head()
df = df[df['종목명'] == 'KB금융']

In [4]:
df_np = df['종가'].values
print(len(df_np))

987


In [5]:
# data split
# input_dim = 50
# forecast_dim = 3

window_size = 50
horizon_size = 3
lr = 0.001
num_epochs = 1000

In [21]:
from sklearn.preprocessing import MinMaxScaler
train_x = []
train_y = []

mm = MinMaxScaler()
fitted = mm.fit(df_np.reshape(-1,1))
out = mm.transform(df_np.reshape(-1,1))
df_np = out.reshape(-1)
for i in range(len(df_np) - window_size - horizon_size):
  train_x.append(df_np[i:i+window_size]) # len == window_size
  train_y.append(df_np[i+window_size: i+window_size + horizon_size]) # len == horizon_size

x_tensor = torch.FloatTensor(train_x)
y_tensor = torch.FloatTensor(train_y)
print(f'src shape : {x_tensor.shape}')
print(f'label shape : {y_tensor.shape}')

src shape : torch.Size([934, 50])
label shape : torch.Size([934, 3])


In [22]:
class Transformer(nn.Module):
  def __init__(self,d_model, n_head, num_enc):
    super(Transformer, self).__init__()

    # encoder layer parameter
    self.d_model = d_model
    self.n_head = n_head
    self.num_enc = num_enc

    self.encoderBlock = nn.TransformerEncoderLayer(
      d_model = self.d_model,
      nhead = self.n_head,
      batch_first = True
    )

    self.encoder = nn.TransformerEncoder(
        encoder_layer = self.encoderBlock,
        num_layers = self.num_enc
    )
    
    self.decoder = nn.Linear(d_model, d_model//2)
    self.fc = nn.Linear(d_model//2, 3)
    self.relu = nn.ReLU()

  def forward(self, x):
      # encoder
    out = self.encoder(x)
      # encoder output == decoder input
      # encoder output shape = encoder input shape

      # decoder
    out = self.decoder(out)
    out = self.relu(out)
    out = self.fc(out)

      # forecast horizon which has a length of 3
    return out

In [23]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [24]:
x_tensor = x_tensor.view(-1,1,window_size).to(device)
y_tensor = y_tensor.view(-1,1,horizon_size).to(device)
print(f'src shape : {x_tensor.shape}')
print(f'label shape : {y_tensor.shape}')

src shape : torch.Size([934, 1, 50])
label shape : torch.Size([934, 1, 3])


In [25]:
# model init
model = Transformer(window_size,2,2).to(device)
optimizer = optim.Adam(model.parameters(), lr = lr)
criterion = nn.MSELoss()

In [26]:
# train_test_split
train_x = x_tensor[:700]
train_y = y_tensor[:700]
test_x = x_tensor[700:]
test_y = y_tensor[700:]

In [28]:
for epoch in range(num_epochs+1):
  optimizer.zero_grad()
  pred = model(train_x)
  loss = criterion(pred, train_y)
  loss.backward()

  optimizer.step()
  if epoch % 100 == 0:
    print(f'Epoch : {epoch} , Loss : {loss.item()}')

Epoch : 0 , Loss : 0.3770117461681366
Epoch : 100 , Loss : 0.005255862604826689
Epoch : 200 , Loss : 0.0033953979145735502
Epoch : 300 , Loss : 0.0026991753838956356
Epoch : 400 , Loss : 0.0024046003818511963
Epoch : 500 , Loss : 0.0021006197202950716
Epoch : 600 , Loss : 0.001969680655747652
Epoch : 700 , Loss : 0.0020828815177083015
Epoch : 800 , Loss : 0.001397137064486742
Epoch : 900 , Loss : 0.0016694514779374003
Epoch : 1000 , Loss : 0.0014558846596628428


In [31]:
model.eval()
y_pred = model(test_x)

eval_loss = criterion(y_pred, test_y)
print(eval_loss.item())

0.0031838417053222656
